<a href="https://colab.research.google.com/github/HaJunYoo/Programmers_DE_Start/blob/main/Transaction_%EB%B0%B0%EC%9A%B0%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 트랜잭션이란? 동시에 Atomic하게 실행되어야 하는 SQL들을 묶어서 하나의 작업처럼 처리하는 방법

In [8]:
# sqlachemy version reinstallation
!pip uninstall sqlalchemy -y
!pip install sqlalchemy==1.4.41

Found existing installation: SQLAlchemy 2.0.0
Uninstalling SQLAlchemy-2.0.0:
  Successfully uninstalled SQLAlchemy-2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.3 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from dotenv import load_dotenv
import os 

path = '/content/drive/MyDrive/Programmers'
# load .env
load_dotenv(os.path.join(path, '.env'))

User = os.environ.get('user')
Password = os.environ.get('password')

In [4]:
import psycopg2

# Redshift connection 함수
# - user와 password를 본인 것으로 수정!

# autocommit이라는 변수가 생김 => True or False를 통해 조절
def get_Redshift_connection(autocommit):
    host = "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
    user = User
    password = Password
    port = 5439
    dbname = "dev"
    conn = psycopg2.connect(f"dbname={dbname} user={user} host={host} password={password} port={port}")
    conn.set_session(autocommit=autocommit)
    return conn

# INSERT SQL을 autocommit=False로 실행

In [5]:
conn = get_Redshift_connection(False) 
# auto_commit = False : 쓰기를 하면 commit을 명시해야 반영이 된다
cur = conn.cursor()

In [6]:
cur.execute("SELECT * FROM hajuny129.name_gender LIMIT 10;")
res = cur.fetchall()
for r in res:
  print(r)

In [7]:
cur.execute("DELETE FROM hajuny129.name_gender;")

In [8]:
cur.execute("INSERT INTO hajuny129.name_gender VALUES ('Keeyong', 'Male');")

In [9]:
cur.execute("SELECT * FROM hajuny129.name_gender LIMIT 10;")
res = cur.fetchall()
for r in res:
  print(r)

('Keeyong', 'Male')


In [10]:
cur.execute("COMMIT;")   
# conn.commit()와 동일. cur.execute("ROLLBACK;")와 conn.rollback()도 동일

In [11]:
conn.close()

# INSERT SQL을 autocommit=False로 실행하고 try/except로 컨트롤하기

In [12]:
conn = get_Redshift_connection(False)
cur = conn.cursor()

In [16]:
# auto_commit = False일때는 BEGIN이 항상 써있는 상태와 같다 => 정합성 이슈 해결
try:
  cur.execute("DELETE FROM hajuny129.name_gender;") 
  cur.execute("INSERT INTO hajuny129.name_gender VALUES ('Claire', 'Female');")
  conn.commit()  # cur.execute("COMMIT;")와 동일
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  conn.rollback()  # cur.execute("ROLLBACK;")와 동일

# finally :
#   conn.close()

# INSERT SQL을 autocommit=True로 실행하고 try/except로 컨트롤하기

In [30]:
conn = get_Redshift_connection(True)
# True 일 때는 바로 physical table에 작성 및 반영을 해버린다.
cur = conn.cursor()

In [31]:
cur.execute("DELETE FROM hajuny129.name_gender;") 

In [32]:
try:
  cur.execute("BEGIN;")
  cur.execute("DELETE FROM hajuny129.name_gender;") 
  cur.execute("INSERT INTO hajuny129.name_gender VALUES ('Claire', 'Female');")
  cur.execute("END;")
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  cur.execute("ROLLBACK;")
# finally :
#   conn.close()

# INSERT SQL을 autocommit=True로 실행하고 SQL로 컨트롤하기

In [33]:
conn = get_Redshift_connection(True)
cur = conn.cursor()

In [34]:
cur.execute("SELECT * FROM hajuny129.name_gender;")
res = cur.fetchall()
for r in res:
  print(r)

('Claire', 'Female')


In [35]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM hajuny129.name_gender;")
cur.execute("INSERT INTO hajuny129.name_gender VALUES ('Benjamin', 'Male');")

In [36]:
cur.execute("SELECT * FROM hajuny129.name_gender;")
res = cur.fetchall()
for r in res:
  print(r)

('Benjamin', 'Male')


In [37]:
cur.execute("END;")

# 잘못된 SQL을 중간에 실행해보기

In [38]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM hajuny129.name_gender;")
cur.execute("INSERT INTO hajuny129.name_gender3 VALUES ('Andrew', 'Male');")
cur.execute("END;")

UndefinedTable: ignored